In [1]:
from transformers import Blip2ForConditionalGeneration, AutoProcessor
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)
model = model.to(device)

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from datasets import Dataset
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
prompt = "How many cats are there?"

encoding = processor(image, prompt, return_tensors="pt")

data_list = [
    {
        'images': image,
        'prompt': prompt,
        'chosen': "two",
        'rejected': 'three',
    }
]

def gen():
    for data in data_list:
        yield data

dataset = Dataset.from_generator(gen)
print("dataset 0 chosen", dataset[0]['chosen'])
print("dataset 0 rejected", dataset[0]['rejected'])
print("dataset 0 prompt", dataset[0]['prompt'])

Generating train split: 0 examples [00:00, ? examples/s]

dataset 0 chosen two
dataset 0 rejected three
dataset 0 prompt How many cats are there?


In [3]:
from trl import DPOConfig, DPOTrainer
from peft import LoraConfig

# Train the model
training_args = DPOConfig(
    output_dir="sadmankiba/blip2-dpo",
    bf16=True,
    gradient_checkpointing=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    logging_steps=1,
    remove_unused_columns=False
)
trainer = DPOTrainer(
    model,
    ref_model=None,  # not needed when using peft
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor,
    peft_config=LoraConfig(),
)

trainer.train()

Extracting prompt from train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

/users/Sadman/venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,0.704900


TrainOutput(global_step=1, training_loss=0.7048635482788086, metrics={'train_runtime': 3.3475, 'train_samples_per_second': 0.299, 'train_steps_per_second': 0.299, 'total_flos': 3263468465553408.0, 'train_loss': 0.7048635482788086, 'epoch': 1.0})